# music-it
Dataset from https://github.com/mdeff/fma <br/>
https://os.unil.cloud.switch.ch/fma/fma_small.zip <br/>

In [ ]:
!pip install pretty_midi
!pip install music21

In [49]:
import glob
import pickle
import random
from datetime import datetime
import pretty_midi
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from music21 import converter, instrument, stream, note, chord

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

is_colab = True
has_dataset = True

In [ ]:
#!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
#!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0.csv
#!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0.json
#!unzip maestro-v3.0.0-midi.zip
#!mv maestro-v3.0.0/ /content/drive/MyDrive/maestro
#!mv maestro-v3.0.0.csv /content/drive/MyDrive/maestro
#!mv maestro-v3.0.0.json /content/drive/MyDrive/maestro
#!rm maestro-v3.0.0-midi.zip

In [ ]:
if is_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    data_dir = "/content/drive/MyDrive/maestro-v3.0.0/**/*.midi"
    tracks_dir = "/content/drive/MyDrive/maestro-v3.0.0/tracks"
    note_to_int_dir = "/content/drive/MyDrive/maestro-v3.0.0/note_to_int"
    int_to_note_dir = "/content/drive/MyDrive/maestro-v3.0.0/int_to_note"
else:
    data_dir = "maestro-v3.0.0/**/*.midi"
    tracks_dir = "maestro-v3.0.0/tracks"
    note_to_int_dir = "maestro-v3.0.0/note_to_int"
    int_to_note_dir = "maestro-v3.0.0/int_to_note"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if not has_dataset:
    count = 0
    for i, file in enumerate(glob.glob(data_dir, recursive=True)):
      count += 1

    tracks = []
    for i, file in enumerate(glob.glob(data_dir, recursive=True)):
      print(f"{i+1}/{count} {file}")
      midi = converter.parse(file)
      pick = None
      for j in midi:
        track = []
        songs = instrument.partitionByInstrument(j)
        for part in songs.parts:
          pick = part.recurse()
          for element in pick:
            if isinstance(element, note.Note):
              track.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
              track.append(".".join(str(n) for n in element.normalOrder))
        tracks.append(track)

    with open(notes_dir, 'wb') as filepath:
      pickle.dump(notes, filepath)

    with open(tracks_dir, 'wb') as filepath:
      pickle.dump(tracks, filepath)

In [73]:
tracks = pickle.load(open(tracks_dir, "rb"))

notes = []
for track in tracks:
    for note in track:
        notes.append(note)
notes = set(notes)

n_vocab = len(notes)
print(f"Vocab size: {n_vocab}")
print(f"Number of tracks: {len(tracks)}")

pitchnames = sorted(set(item for item in notes))

note_to_int = dict((note, number+1) for number, note in enumerate(pitchnames))
note_to_int['[PAD]'] = 0
int_to_note = dict((v,k) for k,v in note_to_int.items())
int_to_note[0] = '[PAD]'

with open(note_to_int_dir, 'wb') as filepath:
  pickle.dump(note_to_int, filepath)

with open(int_to_note_dir, 'wb') as filepath:
  pickle.dump(int_to_note, filepath)


max_len = max(len(x) for x in tracks)

for track in tracks:
  track += ['[PAD]']*(max_len-len(track))

dataset = []
for track in tracks:
  dataset.append([note_to_int[note] for note in track])

dataset = np.array(dataset)
dataset.shape

Vocab size: 3158
Number of tracks: 1276


(1276, 13063)

In [75]:
np.random.shuffle(dataset)
design_set, test_set = train_test_split(dataset, test_size=0.10, random_state=42)
print("design_set:", design_set.shape)
print("test_set:", test_set.shape)

normalizer = Normalizer().fit(design_set)
design_set = normalizer.transform(design_set)
test_set = normalizer.transform(test_set)

train_set, valid_set = train_test_split(design_set, test_size=0.30, random_state=42)
print("train_set:", train_set.shape)
print("valid_set:", valid_set.shape)

design_set: (1148, 13063)
test_set: (128, 13063)
train_set: (803, 13063)
valid_set: (345, 13063)


In [76]:
def sampling(args):
  """Reparameterization trick by sampling from an isotropic unit Gaussian.
  Parameters
  ----------
  args (tensor): mean and log of variance of Q(z|X)

  Returns
  -------
  z (tensor): sampled latent vector
  """
  z_mean, z_log_var = args
  batch = K.shape(z_mean)[0]  # get dimension of mini-batch
  dim = K.int_shape(z_mean)[1]  # get dimension of each z
  # by default, random_normal has mean = 0 and std = 1.0
  epsilon = K.random_normal(shape=(batch, dim))
  return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [82]:
input_dim = train_set.shape[1]
intermediate_dim = 50
batch_size = 128
latent_dim = 2
epochs = 20
lr = 1e-3

inputs = tf.keras.layers.Input(shape=(input_dim,), name='encoder_input')
#h = tf.keras.layers.Dense(intermediate_dim, activation='relu')(inputs)
inputs_rs = tf.keras.layers.Reshape((1,input_dim), input_shape=(input_dim,))(inputs)
h = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(intermediate_dim), merge_mode='concat')(inputs_rs)

z_mean =tf.keras.layers. Dense(latent_dim, activation='linear', name='z_mean')(h)
z_log_var = tf.keras.layers.Dense(latent_dim, name='z_log_var')(h)
z = tf.keras.layers.Lambda(sampling, output_shape=(latent_dim,), name='z')([z_mean, z_log_var])

encoder = tf.keras.models.Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

latent_inputs = tf.keras.layers.Input(shape=(latent_dim,), name='z')
#h = tf.keras.layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
latent_inputs_rs = tf.keras.layers.Reshape((1,latent_dim), input_shape=(latent_dim,))(latent_inputs)
h = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(intermediate_dim), merge_mode='concat')(latent_inputs_rs)

outputs = tf.keras.layers.Dense(input_dim, activation='sigmoid')(h)

decoder = tf.keras.models.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

outputs = decoder(encoder(inputs)[2])

vae = tf.keras.models.Model(inputs, outputs, name='vae')
vae.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 13063)]      0                                            
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 256)          3344384     encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 2)            514         dense_19[0][0]                   
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 2)            514         dense_19[0][0]                   
____________________________________________________________________________________________

In [83]:
reconstruction_loss = tf.keras.losses.mse(inputs, outputs)  # Start with the Mean Squared Error
reconstruction_loss *= input_dim  # we will average later! This is now the "Squared Error"
# Compute the KL divergence, which is our additional regularization term
kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = K.mean(reconstruction_loss + kl_loss)

vae.add_loss(vae_loss)
vae.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr))

In [84]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
print(f"timestamp: {timestamp}")

with tf.device('/GPU:0'):
  history = vae.fit(train_set,
                  epochs=epochs,
                  batch_size=batch_size,
                  validation_data=(valid_set, None),
                  callbacks=[
                    TensorBoard(log_dir=f"/content/drive/MyDrive/music-it/logs/{timestamp}"), 
                    ModelCheckpoint(filepath=f"/content/drive/MyDrive/music-it/checkpoints/{timestamp}.ckpt", monitor='loss', verbose=0, save_best_only=True, mode='min')
                  ])

timestamp: 20211008-135052
Epoch 1/20
7/7 [==============================] - 1s 119ms/step - loss: 3044.2109 - val_loss: 2658.9070
INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


Epoch 2/20
7/7 [==============================] - 0s 25ms/step - loss: 2142.1121 - val_loss: 1327.0114
INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


Epoch 3/20
7/7 [==============================] - 0s 23ms/step - loss: 757.4909 - val_loss: 272.8053
INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


...

INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


Epoch 19/20
7/7 [==============================] - 0s 29ms/step - loss: 27.3036 - val_loss: 25.8593
INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


Epoch 20/20
7/7 [==============================] - 0s 24ms/step - loss: 25.9659 - val_loss: 25.0347
INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/music-it/checkpoints/20211008-135052.ckpt/assets


In [ ]:
%load_ext tensorboard
logdir = f"/content/drive/MyDrive/music-it/logs/{timestamp}"
%tensorboard --logdir {logdir}